In [2]:
from openpyxl import Workbook
from openpyxl import load_workbook
import pandas as pd

url0  = "http://tcg.uis.unesco.org/wp-content/uploads/sites/4/2021/05/SDG4_Benchmark_Tables_2021.xlsx"  # SDG 4 Benchmarks Indicators Data Book (excel)
url1 = "http://tcg.uis.unesco.org/wp-content/uploads/sites/4/2021/05/SDG4_Global_Tables_2021.xlsx"  # SDG 4 Global Data Book by target (excel)
url2 = "http://tcg.uis.unesco.org/wp-content/uploads/sites/4/2021/05/SDG4_Countries_Tables_2021.xlsx" #SDG 4 Country Data Table (excel)

url3 = "http://tcg.uis.unesco.org/wp-content/uploads/sites/4/2020/11/SDG4-Data-tree-specifications-Mapping.xlsx" # SDG 4 Data Tree Specifications - Mapping (excel)
url4 = "ftp://ftp.uis.unesco.org/BDDS/SDG.zip" # SDG 4 Dataset


## set path
path = r"C:\Users\yi_li\Documents\GitHub\UIS_SDG4Vizz"
cleandata  = path + r"\002_data\022_cleandata"
rawdata    = path + r"\002_data\021_rawdata"


## read the whole workbook
## original xlsx files have to be first saved as xls and the resave as xlsx (noted by debug2)
wb = load_workbook(rawdata+r"\SDG4_Countries_Tables_2021_debug2.xlsx",read_only=False)

## 0. SDG4_Benchmark_Tables_2021
# Example sheet look up 
 # example sheet
ws = wb["Benin"] # or whatever worksheet name
print(ws['E10'].value)



2010


In [4]:
# extract all the comments in each cell in the worksheet and save in a list
df_aux = []
for index, row in enumerate(ws.rows):
    row_aux = []
    not_empty = False
    for cell in row:
        if cell.value != None:
            not_empty = True
        
        row_aux.append(cell.comment)
    if not_empty == True:
        df_aux.append(row_aux)

# transform the list of comments into DF
df_comment=pd.DataFrame(df_aux)
df_comment.head(20)

# reformat the DF of comments: dropping the null lines and add variable names
wsdf = pd.DataFrame(ws.values)
df_comment.columns = wsdf.iloc[0].values
df_comment.drop(list(range(0,10)), inplace=True)

In [5]:
df_comment.shape

(362, 15)

In [6]:
## 1. Iterate over all sheets in the workbook to create the Dictionary of comments (named by sheetnames) with variable names
sheetlist = dict()
for sheet,sheet_name in zip(wb.worksheets,wb.sheetnames):
    df_aux = []
    for index, row in enumerate(sheet.rows):
        row_aux = []
        not_empty = False
        for cell in row:
            if cell.value != None:
                not_empty = True
                
            row_aux.append(cell.comment)
        if not_empty ==True:
            df_aux.append(row_aux)
                
    df_comment=pd.DataFrame(df_aux)
    wsdf = pd.DataFrame(sheet.values)
    df_comment.columns = wsdf.iloc[9].values
    df_comment.drop(list(range(0,7)), inplace=True)
    sheetlist[sheet_name]=df_comment
    

In [7]:
sheetlist['Tunisia']

,SDG Target,SDG Table,Related SDG Indicator,SDG Indicator,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,None,None,None,None,None,None,Comment: SOURCE: Programme for International S...,None,None,Comment: SOURCE: Programme for International S...,None,None,None,None,None
9,None,None,None,None,None,None,Comment: SOURCE: Programme for International S...,None,None,Comment: SOURCE: Programme for International S...,None,None,None,None,None
10,None,None,None,None,None,None,Comment: SOURCE: Programme for International S...,None,None,Comment: SOURCE: Programme for International S...,None,None,None,None,None
11,None,None,None,None,None,None,Comment: SOURCE: Programme for International S...,None,None,Comment: SOURCE: Programme for International S...,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
366,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
367,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
368,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [8]:
vdf = []
count=0
for sheet_name, sheet_df in sheetlist.items():
    if sheet_name in ['TOC']:
        continue
    else:
        for rowno, rowval in sheet_df.iterrows():
            for year in range(2010, 2021):
                vlist = []
                vlist.append(rowval["SDG Target"])
                vlist.append(rowval["SDG Table"])
                vlist.append(rowval["Related SDG Indicator"])
                vlist.append(rowval["SDG Indicator"])
                vlist.append(year)
                vlist.append(rowval[str(year)])
                vlist.append(sheet_name)
                vdf.append(vlist)


In [9]:
vdf=pd.DataFrame(vdf)
vdf.columns=["SDG Target", "SDG Table", "Related SDG Indicator", "SDG Indicator", "Year", "Comments", "Country"]

In [10]:
vdf.to_csv(cleandata + r'\SDG4_Country_Data_Table_comments.csv')

In [23]:
vdf.Country.value_counts()['Benin']
vdf

,SDG Target,SDG Table,Related SDG Indicator,SDG Indicator,Year,Comments,Country
0,None,None,None,None,2010,None,Tunisia
1,None,None,None,None,2011,None,Tunisia
2,None,None,None,None,2012,None,Tunisia
3,None,None,None,None,2013,None,Tunisia
4,None,None,None,None,2014,None,Tunisia
...,...,...,...,...,...,...,...
640140,None,None,None,None,2016,None,Netherlands Antilles
640141,None,None,None,None,2017,None,Netherlands Antilles
640142,None,None,None,None,2018,None,Netherlands Antilles
640143,None,None,None,None,2019,None,Netherlands Antilles


In [12]:
datadf = pd.read_csv(cleandata+r'\SDG4_Country_Data_Table.csv',encoding='cp1252')

In [24]:
datadf.Country.value_counts()['Benin']
datadf

,Unnamed: 0,SDG Target,SDG Table,Related SDG Indicator,SDG Indicator,Year,Value,Country,INDICATOR_ID
0,0,1.a,1.a.2,..,Expenditure on education as a percentage of to...,2010,24.84683,Tunisia,XGOVEXP.IMF
1,1,1.a,1.a.2,..,Expenditure on education as a percentage of to...,2011,..,Tunisia,XGOVEXP.IMF
2,2,1.a,1.a.2,..,Expenditure on education as a percentage of to...,2012,20.99101,Tunisia,XGOVEXP.IMF
3,3,1.a,1.a.2,..,Expenditure on education as a percentage of to...,2013,..,Tunisia,XGOVEXP.IMF
4,4,1.a,1.a.2,..,Expenditure on education as a percentage of to...,2014,..,Tunisia,XGOVEXP.IMF
...,...,...,...,...,...,...,...,...,...
640140,640140,4.5,4.5.1,4.2.4,"Gross enrolment ratio, early childhood educati...",2016,..,Netherlands Antilles,GER.01.GPIA
640141,640141,4.5,4.5.1,4.2.4,"Gross enrolment ratio, early childhood educati...",2017,..,Netherlands Antilles,GER.01.GPIA
640142,640142,4.5,4.5.1,4.2.4,"Gross enrolment ratio, early childhood educati...",2018,..,Netherlands Antilles,GER.01.GPIA
640143,640143,4.5,4.5.1,4.2.4,"Gross enrolment ratio, early childhood educati...",2019,..,Netherlands Antilles,GER.01.GPIA


In [25]:
result = pd.concat([datadf, vdf], axis=1).reindex(vdf.index)

In [26]:
result

,Unnamed: 0,SDG Target,SDG Table,Related SDG Indicator,SDG Indicator,Year,Value,Country,INDICATOR_ID,SDG Target,SDG Table,Related SDG Indicator,SDG Indicator,Year,Comments,Country
0,0,1.a,1.a.2,..,Expenditure on education as a percentage of to...,2010,24.84683,Tunisia,XGOVEXP.IMF,None,None,None,None,2010,None,Tunisia
1,1,1.a,1.a.2,..,Expenditure on education as a percentage of to...,2011,..,Tunisia,XGOVEXP.IMF,None,None,None,None,2011,None,Tunisia
2,2,1.a,1.a.2,..,Expenditure on education as a percentage of to...,2012,20.99101,Tunisia,XGOVEXP.IMF,None,None,None,None,2012,None,Tunisia
3,3,1.a,1.a.2,..,Expenditure on education as a percentage of to...,2013,..,Tunisia,XGOVEXP.IMF,None,None,None,None,2013,None,Tunisia
4,4,1.a,1.a.2,..,Expenditure on education as a percentage of to...,2014,..,Tunisia,XGOVEXP.IMF,None,None,None,None,2014,None,Tunisia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640140,640140,4.5,4.5.1,4.2.4,"Gross enrolment ratio, early childhood educati...",2016,..,Netherlands Antilles,GER.01.GPIA,None,None,None,None,2016,None,Netherlands Antilles
640141,640141,4.5,4.5.1,4.2.4,"Gross enrolment ratio, early childhood educati...",2017,..,Netherlands Antilles,GER.01.GPIA,None,None,None,None,2017,None,Netherlands Antilles
640142,640142,4.5,4.5.1,4.2.4,"Gross enrolment ratio, early childhood educati...",2018,..,Netherlands Antilles,GER.01.GPIA,None,None,None,None,2018,None,Netherlands Antilles
640143,640143,4.5,4.5.1,4.2.4,"Gross enrolment ratio, early childhood educati...",2019,..,Netherlands Antilles,GER.01.GPIA,None,None,None,None,2019,None,Netherlands Antilles


In [27]:
result = result.loc[:,~result.columns.duplicated()]

In [28]:
result

,Unnamed: 0,SDG Target,SDG Table,Related SDG Indicator,SDG Indicator,Year,Value,Country,INDICATOR_ID,Comments
0,0,1.a,1.a.2,..,Expenditure on education as a percentage of to...,2010,24.84683,Tunisia,XGOVEXP.IMF,None
1,1,1.a,1.a.2,..,Expenditure on education as a percentage of to...,2011,..,Tunisia,XGOVEXP.IMF,None
2,2,1.a,1.a.2,..,Expenditure on education as a percentage of to...,2012,20.99101,Tunisia,XGOVEXP.IMF,None
3,3,1.a,1.a.2,..,Expenditure on education as a percentage of to...,2013,..,Tunisia,XGOVEXP.IMF,None
4,4,1.a,1.a.2,..,Expenditure on education as a percentage of to...,2014,..,Tunisia,XGOVEXP.IMF,None
...,...,...,...,...,...,...,...,...,...,...
640140,640140,4.5,4.5.1,4.2.4,"Gross enrolment ratio, early childhood educati...",2016,..,Netherlands Antilles,GER.01.GPIA,None
640141,640141,4.5,4.5.1,4.2.4,"Gross enrolment ratio, early childhood educati...",2017,..,Netherlands Antilles,GER.01.GPIA,None
640142,640142,4.5,4.5.1,4.2.4,"Gross enrolment ratio, early childhood educati...",2018,..,Netherlands Antilles,GER.01.GPIA,None
640143,640143,4.5,4.5.1,4.2.4,"Gross enrolment ratio, early childhood educati...",2019,..,Netherlands Antilles,GER.01.GPIA,None


In [29]:
result.to_csv(cleandata+r'\final_SDG4_Country_Data_Table.csv')